In [ ]:
%pip install plotly
%pip install --upgrade nbformat

## Building an AI Travel Assistant For Global Explorers

Watch Dev Day Demo 33:33
https://www.youtube.com/watch?v=U9mJuUkhUzk

Get MapBox Access Token:
https://www.mapbox.com/

In [1]:
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv
import os

_: bool = load_dotenv(find_dotenv())  # read local .env file

client: OpenAI = OpenAI()

MAPBOX_ACCESS_TOKEN = os.environ.get("MAPBOX_TOKEN")

### Step 0-A Seed Configuration For Assistant

In [2]:
from typing import Union

# Because dictionaries are mutable, any changes you make to a dictionary are done in place.
# Here we are them to store the state of map. It allows to retrieve the state and easily update it using function calling.

# When you use the global keyword in a function, it allows you to access and modify variables that are defined in the global scope from within the function. 

map_state: dict[str, Union[float, str]] = {
            "latitude": 39.949610,
            "longitude": -75.150282,
            "zoom": 16,}

markers_state: dict[str, list[Union[float, str]]] = {
    "latitudes": [],
    "longitudes": [],
    "labels": [],
}

# Define Functions For Our Assistant - Use Try Catch Blocks For Error Handling

# Function 1 - Control Map Location
def update_map(longitude: float, latitude: float, zoom: int) -> str:
    """Update map to center on a particular location."""

    global map_state  # Refer to the global map_state

    try: 
        if not (-90 <= latitude <= 90) or not (-180 <= longitude <= 180) or not (isinstance(zoom, int) and zoom > 0):
            raise ValueError("""Latitude must be between -90 and 90,
                             Longitude must be between -180 and 180,
                             and Zoom must be a positive integer.""")

        # Update the global map_state
        map_state['latitude'] = latitude
        map_state['longitude'] = longitude
        map_state['zoom'] = zoom

        return "Map Updated."
    except (ValueError, TypeError) as e:
        raise ValueError(f"Error in update_map function: {e}")

# Function 2 - Add markers to map
def add_markers(latitudes: list, longitudes: list, labels: list) -> str:
    """OpenAI tool to update markers in-app"""

    global markers_state  # Refer to the global map_state

    try: 
         # Validate inputs
        if len(latitudes) != len(longitudes) or len(latitudes) != len(labels):
            raise ValueError("Number of latitudes, longitudes, and labels must be the same.")

        markers_state["latitudes"] = latitudes
        markers_state["longitudes"] = longitudes
        markers_state["labels"] = labels
        
        return "Markers added successfully."
    
    except (ValueError, TypeError) as e:
        raise ValueError(f"Error in add_markers function: {e}")


available_functions = {
    "update_map": update_map,
    "add_markers": add_markers,
}

SEED_INSTRUCTION: str = """You are a reliable travel assistant, here to assist Global Explorers in planning and discovering their next travel 
                            destination. You specialize in marking locations on the map, providing personalized travel suggestions, and helping 
                            you reach any destination you have in mind. Wherever possible mark locations on map while making the travelers travel 
                            plans memorable. In map marker label share the destination names """


travel_agent_tools = [
    {
        "type": "function",
        "function": {
            "name": "update_map",
            "description": "Update map to center on a particular location",
            "parameters": {
                "type": "object",
                "properties": {
                    "longitude": {
                        "type": "number",
                        "description": "Longitude of the location to center the map on"
                    },
                    "latitude": {
                        "type": "number",
                        "description": "Latitude of the location to center the map on"
                    },
                    "zoom": {
                        "type": "integer",
                        "description": "Zoom level of the map"
                    }
                },
                "required": ["longitude", "latitude", "zoom"]
            }
        }
    },
    {
        "type": "function",
        "function":  {
            "name": "add_markers",
            "description": "Add list of markers to the map",
            "parameters": {
                "type": "object",
                "properties": {
                    "longitudes": {
                        "type": "array",
                        "items": {
                            "type": "number"
                        },
                        "description": "List of longitude of the location to each marker"
                    },
                    "latitudes": {
                        "type": "array",
                        "items": {
                            "type": "number"
                        },
                        "description": "List of latitude of the location to each marker"
                    },
                    "labels": {
                        "type": "array",
                        "items": {
                            "type": "string"
                        },
                        "description": "List of text to display on the location of each marker"
                    }
                },
                "required": ["longitudes", "latitudes", "labels"]
            }
        }
    },

]

#### Step 0-B Function To Plot and Update the Map

In [3]:
import plotly.graph_objects as go

# Function to create and display the map
def create_map(map_state: dict[str, Union[float, str]], markers_state: dict[str, list[Union[float, str]]]) -> None:
    
    figure = go.Figure(go.Scattermapbox(mode="markers"))
    
    figure.add_trace(
        go.Scattermapbox(
            mode="markers",
            marker=dict(color='red', size=14),
            lat=markers_state["latitudes"],
            lon=markers_state["longitudes"],
            text=markers_state["labels"]
        )
    )
    
    figure.update_layout(
        mapbox=dict(
            accesstoken=MAPBOX_ACCESS_TOKEN, # If you don't have MAPBOX token, replace with style="open-street-map".
            center=go.layout.mapbox.Center(
                lat=map_state["latitude"],
                lon=map_state["longitude"]
            ),
            zoom=map_state["zoom"]
        ),
        margin=dict(l=0, r=0, t=0, b=0)
    )
    figure.show()

### Step 01 - Create an Assistant

In [4]:
from openai.types.beta.assistant import Assistant

travel_agent: Assistant = client.beta.assistants.create(
    model="gpt-3.5-turbo-1106",
    name="Travel Agent",
    instructions=SEED_INSTRUCTION,
    tools=travel_agent_tools,
)

### Step 02 - Create a Thread

In [5]:
from openai.types.beta.thread import Thread

thread: Thread = client.beta.threads.create()

### Step 03 - Add Messages to Thread

In [6]:
from openai.types.beta.threads import ThreadMessage

add_message_to_thread: ThreadMessage = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I will visit Singapore this december. Can you share 2 must visit travel destinations that are next to each other and show them on map?",
)

### Step 04 - Create a Run

In [7]:
from openai.types.beta.threads import Run

run: Run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=travel_agent.id
)

### Step 05: Polling for Updates and Calling Functions

In [8]:
## Helpful Utility Functions to do parallel function calling

from openai.types.beta.threads import Run 
from openai.types.beta.thread import Thread 
import time
import json

# Polling 
def wait_for_completion(run: Run, thread: Thread):

        if run is None:
            raise ValueError("Run is not set!")

        # while run.status in ["in_progress", "queued"]:
        while run.status not in ["completed", "failed"]:
            run_status = client.beta.threads.runs.retrieve(
                thread_id=thread.id,
                run_id=run.id
            )
            time.sleep(3)  # Wait for 3 seconds before checking again
            print(f"Status: {run_status.status}")

            if run_status.status == 'completed':
                print("Run completed.")
                return client.beta.threads.messages.list(thread_id=thread.id)
                # break
            elif run_status.status == 'requires_action' and run_status.required_action is not None:
                print(f"Function Calling ...")
                call_required_functions(
                    toolCalls=run_status.required_action.submit_tool_outputs.model_dump(),
                    thread_id=thread.id,
                    run_id=run.id
                    )
            elif run.status == "failed":
                print("Run failed.")
                break
            else:
                print(f"Waiting for the Assistant to process...: {run.status}")

# Function to call the required functions
def call_required_functions(toolCalls, thread_id: str, run_id: str):
        
        tool_outputs = []

        # for toolcall in toolCalls:
        for toolcall in toolCalls["tool_calls"]:
            function_name = toolcall['function']['name']
            function_args = json.loads(toolcall['function']['arguments'])
            
            if function_name in available_functions:

                # Displaying the message with values
                print(f"calling function {function_name} with args:")
                for key, value in function_args.items():
                    print(f"{key}: {value}")

                if function_name in available_functions:
                    function_to_call = available_functions[function_name]
                    output = function_to_call(**function_args)

                    print("Output Status", output)
                    
                    tool_outputs.append({
                        "tool_call_id": toolcall['id'],
                        "output": output,
                    })

            else:
                raise ValueError(f"Unknown function: {function_name}")               
 
        print('submit_tool_outputs >>>>>' ,tool_outputs,) 
        # Submit tool outputs and update the run
        client.beta.threads.runs.submit_tool_outputs(
            thread_id=thread_id,
            run_id=run_id,
            tool_outputs=tool_outputs
            )


In [9]:
final_response = wait_for_completion(thread=thread, run=run)

Status: requires_action
Function Calling ...
calling function update_map with args:
longitude: 103.8535
latitude: 1.2833
zoom: 12
Output Status Map Updated.
calling function add_markers with args:
longitudes: [103.8198, 103.8223]
latitudes: [1.3329, 1.3166]
labels: ['Gardens by the Bay', 'Marina Bay Sands']
Output Status Markers added successfully.
submit_tool_outputs >>>>> [{'tool_call_id': 'call_npFCOOWb8k4CzscyFIdjjd4i', 'output': 'Map Updated.'}, {'tool_call_id': 'call_ELKMcec9PebkGSm71rCLixJ2', 'output': 'Markers added successfully.'}]
Status: in_progress
Waiting for the Assistant to process...: queued
Status: completed
Run completed.


In [10]:
# verify the map_state and markers_state
print(markers_state)
print(map_state)

{'latitudes': [1.3329, 1.3166], 'longitudes': [103.8198, 103.8223], 'labels': ['Gardens by the Bay', 'Marina Bay Sands']}
{'latitude': 1.2833, 'longitude': 103.8535, 'zoom': 12}


In [11]:
for m in (final_response.data):
    (print(m.role,":", ), print(m.content[0].text.value))
    print()

create_map(map_state=map_state, markers_state=markers_state)

assistant :
Great choice! Two must-visit travel destinations in Singapore that are next to each other are "Gardens by the Bay" and "Marina Bay Sands." I've marked these locations on the map for you. Enjoy your trip to Singapore!

user :
I will visit Singapore this december. Can you share 2 must visit travel destinations that are next to each other and show them on map?



# Take It to Next Step with these Simple Challenge: 
1. Extract All assistant functions in a seperate Class. 
2. Create assistant once in a seperate file names seed_assistant.py. Store assistant_id in an env variable. Now using the above class Use Assistant Id to retrive the assistant.
3. Now in this notebook create a function that will take user prompt, add messages to thread, create run, do polling and display the final response like above
4. Add Retrival and Code Inteprator to add features just like in dev day demo.